# YAML Sandbox

Sandbox notebook to tune up using YAML with the MODS azcam server

YAML (YAML Ain't Markup Language) is a simple, structured, ascii-based data 
serialization language used across many  languages and operating systems that 
is often used for human-readable and editable runtime configuration files because 
of its simple syntax.

### Sources

 * Official YAML website: https://yaml.org/
 * PyYAML, the standard python YAML loader/emitter (comes with Anaconda): http://pyyaml.org 
 * A nice YAML tutorial: https://www.cloudbees.com/blog/yaml-tutorial-everything-you-need-get-started
 * Another YAML tutorial: https://python.land/data-processing/python-yaml


In [27]:
import os

# YAML module

import yaml

# proper paths

from pathlib import Path

## Load and parse a YAML file

### loadYAML() function

We use the yaml `safe_load()` method instead of the deprecated `load()` method.

In [28]:
def loadYAML(fileName):
    """
    Load the contents of a YAML-format file using safe_load()
    """
    
    if os.path.exists(fileName):

        try:
            stream = open(fileName,'r')
        except Exception as exp:
            raise RuntimeError(f"ERROR: could not open {fileName}: {exp}")
            
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exp:
            raise RuntimeError(f"ERROR: could not load {fileName}: {exp}")
        
        if len(data)==0:
            raise RuntimeError(f"YAML file {fileName} is empty!")
        
        return data
    else:
        raise ValueError(f"YAML file {fileName} does not exist")
        

### read and print out

Note that pyYAML strips out all comments (block and inline), so this is a formatted printout of the
file contents without the comments.

In [29]:
myFile = "azcam.ini"

try:
    cfgData = loadYAML(myFile)
    print(f"\nRead YAML file {myFile}, contents:")
    for key in cfgData:
        print(f"\n{key}:")
        yamlItem = cfgData[key]
        for keyword in yamlItem:
            print(f"  {keyword}: {yamlItem[keyword]}")
except Exception as exp:
    print(f"ERROR: loadYAML() - {exp}")



Read YAML file azcam.ini, contents:

server:
  mode: archon
  system: MODS1B
  rootDir: /home/dts/azcam
  dataDir: /home/data
  logDir: /home/Logs/azcam
  logFile: server.log
  cmdPort: 2402
  webPort: 2403
  useInstrument: T
  useTelescope: T
  verbosity: 2

controller:
  timingFile: MODS1B.ncf
  host: 10.0.0.2
  port: 4242
  resetFlag: 0
  verbosity: 2

tempcon:
  useHeater: T
  board: MOD10
  tempIDs: [0, 1]
  setPoint: -100.0

exposure:
  shutterDelay: 250
  fileType: MEF
  useDisplay: F

telescope:
  lbtSide: left


## check interpretation

In [34]:
# Server config

modsID = cfgData["server"]["system"]
azcam_db_systemname = modsID

azcam_db_rootfolder = str(Path(cfgData["server"]["rootDir"]))
azcam_db_datafolder = str(Path(cfgData["server"]["dataDir"]))

azcam_db_verbosity = cfgData["server"]["verbosity"]

if cfgData["server"]["useTelescope"]:
    telescope_is_enabled=1
else:
    telescope_is_enabled=0

if cfgData["server"]["useInstrument"]:
    instrument_is_enabled=1
else:
    instrument_is_enabled=0

logfile = str(Path(cfgData["server"]["logDir"]) / cfgData["server"]["logFile"])

azcam_db_servermode = cfgData["server"]["mode"]

cmdport = cfgData["server"]["cmdPort"]
lbtSide = cfgData["telescope"]["lbtSide"]

print("azcam server config:")
print(f"  Mode: {azcam_db_servermode}")
print(f"  MODS: {modsID}")
print(f"  side: {lbtSide}")
print(f"  rootDir: {azcam_db_rootfolder}")
print(f"  dataDir: {azcam_db_datafolder}")
print(f"  logFile: {logfile}")
print(f"  cmdPort: {cmdport}")
if telescope_is_enabled:
    print("  telescope: enabled")
else:
    print("  telescope: disabled")
if instrument_is_enabled:
    print(" instrument: enabled")
else:
    print(" instrument: disabled")
print(f"  verbosity: {azcam_db_verbosity}")

# Controller

controller_timing_file = cfgData["controller"]["timingFile"]
controller_camserver_host = cfgData["controller"]["host"]
controller_camserver_port = cfgData["controller"]["port"]
controller_reset_flag = cfgData["controller"]["resetFlag"]
controller_verbosity = cfgData["controller"]["verbosity"]

print("\nArchon controller config:")
print(f"  Timing File: {controller_timing_file}")
print(f"  Archon Host: {controller_camserver_host}:{controller_camserver_port}")
print(f"   Reset Flag: {controller_reset_flag}")
print(f"    verbosity: {controller_verbosity}")

# Temperature Control

print("\nTemperature control:")
if (cfgData["tempcon"]["useHeater"]):
    print(f"       Board: {cfgData['tempcon']['board']}")
    print(f"  Sensor IDs: {cfgData['tempcon']['tempIDs']}")
    print(f"   Set Point: {cfgData['tempcon']['setPoint']} C")
else:
    print("  None")

# Exposure

print("\nExposure config:")
print(f'  Shutter delay: {cfgData["exposure"]["shutterDelay"]} msec')
print(f'      File type: {cfgData["exposure"]["fileType"]}')


azcam server config:
  Mode: archon
  MODS: MODS1B
  side: left
  rootDir: /home/dts/azcam
  dataDir: /home/data
  logFile: /home/Logs/azcam/server.log
  cmdPort: 2402
  telescope: enabled
 instrument: enabled
  verbosity: 2

Archon controller config:
  Timing File: MODS1B.ncf
  Archon Host: 10.0.0.2:4242
   Reset Flag: 0
    verbosity: 2

Temperature control:
       Board: MOD10
  Sensor IDs: [0, 1]
   Set Point: -100.0 C

Exposure config:
  Shutter delay: 250 msec
      File type: MEF
